In [0]:
import os
import pandas as pd
import numpy as np
import timeit
import ujson
#Usamos ujson que parece que es el que hace la importación más rápida. Añadir la documentación donde se comenta.
#from google.colab import drive  #Lo pongo más abajo, porque si no me da error (por qué??!!)

In [2]:
!ls -lh

total 4.0K
drwxr-xr-x 2 root root 4.0K Oct 22 18:24 sample_data


##IMPORTACIÓN DESDE GOOGLE DRIVE

In [0]:
#Esto sólo funciona en Google Colab porque tiene una librería de Google Colab.
#Más documentación en:
#https://colab.research.google.com/notebooks/io.ipynb
#en el apartado PyDrive.
#PyDrive documentación:
#https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz

In [0]:
#Estos ficheros están en mi google drive.

#Vínculo a la carpeta comprimida con todas las pelis en mi Google Drive:
#https://drive.google.com/open?id=1-jrX_p1VfgMceGIj7SK_eIef8syIGsdL

#movies
file_id = '1-jrX_p1VfgMceGIj7SK_eIef8syIGsdL'

downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("movies.zip")

#trial
#file_id = '1ivGsdMnbPyHuxXPoZAnQVnjKp82-hq9o'

#downloaded = drive.CreateFile({'id': file_id})
#downloaded.GetContentFile("trial.zip")

In [38]:
!ls -lh

total 1.5G
-rw-r--r-- 1 root root 2.5K Oct 24 15:15 adc.json
drwx------ 3 root root 4.0K Oct 24 15:19 drive
-rw-r--r-- 1 root root 1.5G Oct 24 15:15 movies.zip
drwxr-xr-x 2 root root 4.0K Oct 22 18:24 sample_data
drwxr-xr-x 3 root root 4.0K Oct 24 15:16 themoviedb_data


In [6]:
#Da un error pero no tiene consecuencias

#!mkdir themoviedb_data
#!cd themoviedb_data && unzip -q ../trial.zip
#!ls -hl themoviedb_data

!mkdir themoviedb_data
!cd themoviedb_data && unzip -q ../movies.zip
!ls -hl themoviedb_data

warning [../movies.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [../movies.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #409792).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)
total 12M
drwxr-xr-x 2 root root 12M Oct 24 15:18 movies


In [7]:
!ls -hl ./themoviedb_data/movies | head -10

total 4.3G
-rw-r--r-- 1 root root 7.4K Aug 26 18:10 100009.json
-rw-r--r-- 1 root root  22K Aug 25 12:44 10000.json
-rw-r--r-- 1 root root  24K Aug 26 18:10 100010.json
-rw-r--r-- 1 root root  18K Aug 26 18:10 100011.json
-rw-r--r-- 1 root root  16K Aug 26 18:10 100013.json
-rw-r--r-- 1 root root 6.8K Aug 26 18:10 100016.json
-rw-r--r-- 1 root root  19K Aug 26 18:10 100017.json
-rw-r--r-- 1 root root  32K Aug 25 12:44 10001.json
-rw-r--r-- 1 root root  16K Aug 26 18:10 100022.json


In [8]:
!ls -1 themoviedb_data/movies | wc -l

409791


##ACCESO A MI DRIVE

In [9]:
#Esto me va a permitir tener acceso a las carpetas de mi drive desde Colab:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
!ls "/content/drive/My Drive/" -lh

total 4.4G
drwx------ 2 root root  4.0K Jul 14 07:08  20180714_Deep_Learning
-rw------- 1 root root   76K Oct 22 18:13  6885.json
drwx------ 2 root root  4.0K Jun 26  2016 'avast! Files (XT1068, UWbYzWPVmAmwlrlD)'
drwx------ 2 root root  4.0K Jul 18 18:38 'Colab Notebooks'
-rw------- 1 root root   44K Feb  4  2018  contacts.vcf
-rw------- 1 root root  170M Oct 22 12:05  dfMoviesCast.csv
-rw------- 1 root root     1 Apr  9  2018  facturaIVA.gsheet
-rw------- 1 root root     1 Oct 24 13:16 'Mis cuentas.gsheet'
-rw------- 1 root root 1019M Oct 14 16:41  movies.csv
-rw------- 1 root root  1.5G Oct 12 15:09  movies.zip
-rw------- 1 root root     1 Oct 20 08:40 'Nuestras Cuentas.gsheet'
-rw------- 1 root root  1.8G Sep 24 21:07  people.zip
-rw------- 1 root root     1 Jul 24 18:17 'Propuesta de una idea.gslides'
drwx------ 2 root root  4.0K Apr  7  2015  Screencastify
-rw------- 1 root root     1 Sep 30  2017  Series.gdoc
-rw------- 1 root root     1 Jan  2  2016 'Trabajos Iñigo.gsheet'
-rw

## ESTE BLOQUE CONTIENE EL CÓDIGO QUE HE DE EJECUTAR PARA ESTRUCTURAR MIS DATOS:

EL DETALLE DE CÓMO HE LLEGADO A ESTE CÓDIGO ESTÁ EN EL NOTEBOOK 03_TIDYING_MOVIES_DATA.IPYNB

In [49]:
#Lo importo todo. Tengo que reducir el número de campos para que me quepa en memoria.
#Excluyo: backdrop_path, external_ids, homepage, images_backdrops, images_posters, poster_path, similar_movies_page/results/total_pages/total_results,
#video, recommendations_page/results/total_pages/total_results
#Recommendations: películas recomendadas para una película. No es lo mismo que similar_movies.
#Para hacer esto debería hacer una importación inicial de todos los campos y luego, una vez decidido, importo todo.

#Usamos un generator, que es más eficiente en cuanto a memoria. "yield" va asociado al generator.
#Poner la nota: importo todo, aunque sé que lo voy a usar, por si alguien puede querer usarlo.
#Desanido el primer nivel de diccionarios. Para belongs_to_collection, que a veces es nulo en lugar de un diccionario, tomo dos keys.
#Me queda tratar las listas que contienen diccionarios. 
#¿Qué pasa con UTF8? !!!!!!!!!!!!!!!!!!!!!!!!!!
#No vale de nada cambiar el orden!!! Así que mantengo el alfabético del csv
#Por mi código anterior ya sé que adult es un booleano y que no tiene nulos. Así que puedo filtrarlo por adult = False. Voy a excluir estas
#pelis de mi análisis.

path = "./themoviedb_data/movies"
dfMovies = pd.DataFrame()

def data_iterator(path):
    intI = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if intI <= 500000:
                if f.endswith('.json'):
                    fp = os.path.join(root,f)
                    with open(fp) as o:
                        data = ujson.load(o)
                
                    if intI % 10000 == 0:
                        print(intI)
                    
                    result = {"titles": data["alternative_titles"]["titles"], 
                       "budget": data["budget"], "credits_cast": data["credits"]["cast"], "credits_crew": data["credits"]["crew"],
                       "genres": data["genres"], "id": data["id"], "imdb_id": data["imdb_id"],
                       "keywords": data["keywords"]["keywords"], "original_language": data["original_language"],
                       "original_title": data["original_title"], "overview": data["overview"],
                       "popularity": data["popularity"], 
                       "production_companies": data["production_companies"], "production_countries": data["production_countries"],
                       "release_date": data["release_date"], "release_dates": data["release_dates"]["results"],
                       "revenue": data["revenue"], 
                       "reviews_page": data["reviews"]["page"],
                       "reviews_results": data["reviews"]["results"],
                       "reviews_total_pages": data["reviews"]["total_pages"],
                       "reviews_total_results": data["reviews"]["total_results"],
                       "runtime": data["runtime"],
                       "spoken_languages": data["spoken_languages"], "status": data["status"],
                       "tagline": data["tagline"], "title": data["title"], 
                       "vote_average": data["vote_average"], "vote_count": data["vote_count"]}
                    
                    if data["belongs_to_collection"] is None:
                        result["belongs_to_collection_id"] = None
                        result["belongs_to_collection_name"] = None
                    else:
                        result["belongs_to_collection_id"] = data["belongs_to_collection"]["id"]
                        result["belongs_to_collection_name"] = data["belongs_to_collection"]["name"]
                    
                    if data['adult'] == False:
                        yield result
            else:
                break          

tic=timeit.default_timer()
dfMovies = pd.DataFrame(data_iterator(path))
dfMovies.set_index('id', inplace=True)
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


319.2362025020011

In [0]:
#columnName: columna de dfMovies a tratar. Sus elemeentos son listas de diccionarios.
#args: lista variable de claves con las que me voy a quedar de los diccionarios.
#Si un elemento es una lista vacía, no hay problema.
def columnIterator(columnName, *args):
    for id in dfMovies.index:
        for dict in dfMovies[dfMovies.index == id][columnName].iloc[0]:
            outputDict = {'movie_id': id}
            for key in args:
                outputDict[key] = dict[key]
            yield outputDict

In [0]:
#Creo mi dataframe con la info de la columna correspondiente, compruebo que tiene el nº filas que ha de tener y, si es así, elimino la columna
#correspondiente en dfMovies. Haré todo esto con la siguiente función:
def columnProcesses (columnName, *args):
    #I need to declare dfMovies as global as I will change it inside this function.
    global dfMovies
    
    dfOutput = pd.DataFrame()
    dfOutput = pd.DataFrame(columnIterator(columnName, *args))
    
    #Compruebo que tiene el nº de registros que ha de tener.
    dfCheck = pd.DataFrame(dfMovies[columnName].map(len).value_counts())
    dfCheck['calculation']=dfCheck[columnName]*dfCheck.index
    
    if dfOutput.shape[0] == dfCheck['calculation'].sum():
        dfMovies = dfMovies.drop(columnName, axis=1)
        return dfOutput
    else:
        return "Error in process"

In [0]:
def columnIteratorReleaseDates(*args):
    for id in dfMoviesReleaseDates1.index:
        for dict in dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['release_dates'].iloc[0]:
            outputDict = {'movie_id': dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['movie_id'].iloc[0],
                         'iso_3166_1': dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['iso_3166_1'].iloc[0]}
            for key in args:
                outputDict[key] = dict[key]
            yield outputDict

In [0]:
#Generación de todas las tablas de una tacada: 
#This module took about 70min to be executed in Google Colab.
tic=timeit.default_timer()

dfMoviesCast = columnProcesses ('credits_cast', 'cast_id', 'character', 'gender', 'id', 'name', 'order')
print("credits_cast generated")
dfMoviesCrew = columnProcesses ('credits_crew', 'department', 'gender', 'id', 'job', 'name')
print("credits_crew generated")
dfMoviesGenres = columnProcesses ('genres', 'id', 'name')
print("genres generated")
dfMoviesKeywords = columnProcesses ('keywords', 'id', 'name')
print("keywords generated")
dfMoviesProductionComp = columnProcesses ('production_companies', 'id', 'name', 'origin_country')
print("production_companies generated")
dfMoviesProductionCountries = columnProcesses ('production_countries', 'iso_3166_1', 'name')
print("production_countries generated")
dfMoviesReviewsResults = columnProcesses ('reviews_results', 'author', 'content', 'id')
print("reviews_results generated")
dfMoviesSpokenLang = columnProcesses ('spoken_languages', 'iso_639_1', 'name')
print("spoken_languages generated")
dfMoviesReleaseDates1 = columnProcesses ('release_dates', 'iso_3166_1', 'release_dates')
print("release_dates 1 generated")
dfMoviesReleaseDates = pd.DataFrame(columnIteratorReleaseDates('certification', 'iso_639_1', 'release_date', 'type'))
print("release_dates generated")
#Libero la memoria
dfMoviesReleaseDates1 = None

toc=timeit.default_timer()
toc - tic #elapsed time in seconds

In [73]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'credits_cast', 'credits_crew', 'genres', 'imdb_id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'release_dates', 'revenue', 'reviews_page', 'reviews_results',
       'reviews_total_pages', 'reviews_total_results', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'titles',
       'vote_average', 'vote_count'],
      dtype='object')

##EXPLORATORY DATA ANALYSIS FOR MOVIES

In [13]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'credits_cast', 'credits_crew', 'genres', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'release_dates', 'revenue', 'reviews_page', 'reviews_results',
       'reviews_total_pages', 'reviews_total_results', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'titles',
       'vote_average', 'vote_count'],
      dtype='object')

In [0]:
dfMovies.sample(3)

####Intento hacerlo con altair pero me encuentro con que hay un límite para 5mil registros.¿Quizá puedo primero agregar antes de representar?

In [0]:
#Intento hacerlo con altair pero me encuentro con que hay un límite para 5mil registros.
#¿Quizá puedo primero agregar antes de representar?

In [0]:
!pip install -U altair vega_datasets notebook vega

In [0]:
import altair as alt
from vega_datasets import data

In [0]:
#Si estoy en un notebook normal, ejecutaría:
#alt.renderers.enable("notebook")

#Pero en colab, ejecuto:
alt.renderers.enable("colab")

In [0]:
#Y aquí tengo el error.
alt.Chart(dfMovies).mark_bar().encode(
   #Como quiero hacer una modificación de x, tengo que incluir alt.X. Si fuese de y, sería alt.Y 
   x=alt.X("status", bin=True),
   y="count()" 
)

####Así que de momento lo hago con matplotlib o seaborn

In [0]:
#Si uso el código de Toni:
import matplotlib.pyplot as plt
%matplotlib inline  
#Para que nos haga los plot en la misma celda y que no salga una nueva pantalla
plt.style.use('ggplot')   
#Esto es para el estilo, pero da un poco igual

In [0]:
#Si uso seaborn, de los cursos de DataCamp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  
sns.set()
#Esto es para el estilo, pero da un poco igual

####EXPLORACIÓN INICIAL:

In [14]:
#Veo qué columnas tienen algún nulo:
dfMovies.isnull().any()

belongs_to_collection_id       True
belongs_to_collection_name     True
budget                        False
credits_cast                  False
credits_crew                  False
genres                        False
keywords                      False
original_language             False
original_title                False
overview                      False
popularity                    False
production_companies          False
production_countries          False
release_date                  False
release_dates                 False
revenue                       False
reviews_page                  False
reviews_results               False
reviews_total_pages           False
reviews_total_results         False
runtime                        True
spoken_languages              False
status                        False
tagline                       False
title                         False
titles                        False
vote_average                  False
vote_count                  

In [15]:
dfMovies['status'].value_counts()

Released           387266
Post Production      2161
In Production        1262
Rumored              1230
Planned              1203
Canceled              117
Name: status, dtype: int64

In [16]:
#No entiendo cómo tienen ingresos las películas que no han sido "Released"
#Como revenue no tiene nulos, puedo contarlo para conocer el nº de registros.
dfMovies.groupby('status')['revenue'].agg(['sum','count'])

,sum,count
status,,
Canceled,186708860,117
In Production,1514636,1262
Planned,1,1203
Post Production,31336004,2161
Released,7395865752226,387266
Rumored,50653894,1230


In [0]:
#Debería quedarme en mi modelo sólo con las "Released". Podría después aplicarlo al resto.

In [18]:
#Me interesa ver de las pelis de antes cuántas tienen revenue=0 (para ver si son solo unas pocas pelis las que producen ingreso cuando status!=Released)
dfMovies[dfMovies['revenue']!=0].groupby('status')['revenue'].agg(['sum','count'])

,sum,count
status,,
Canceled,186708860,1
In Production,1514636,7
Planned,1,1
Post Production,31336004,9
Released,7395865752226,9986
Rumored,50653894,11


In [19]:
dfMovies.shape[0]

393239

In [20]:
dfMovies[dfMovies['revenue']!=0].shape[0]

10015

In [22]:
dfMovies[dfMovies['revenue']>50].shape[0]

9654

In [23]:
#En principio, registros que tendría para mi modelo. ¿Excluiría las pelis con ingresos >0 y <50? Habría que echar un vistazo.
#¿Excluiría las pelis de este año? Porque quizás los datos de ingresos no se han estabilizado.
dfMovies[dfMovies['revenue']>50][dfMovies['status']=='Released'].shape[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


9628

In [24]:
#¿De cuántas pelis tengo el presupuesto?
dfMovies[dfMovies['budget']>50][dfMovies['status']=='Released'].shape[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


16937

In [31]:
#¿De cuántaas pelis coincide que tengo presupuesto e ingresos?
dfMovies[dfMovies['revenue']>50][dfMovies['budget']>50][dfMovies['status']=='Released'].shape[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


6556

In [30]:
#Relajando el criterio de >50:
dfMovies[(dfMovies['revenue']>0) & (dfMovies['budget']>0) & (dfMovies['status']=='Released')].shape[0]

6823

In [37]:
dfMovies[dfMovies['title']=='Avatar'][['release_date','credits_cast']]

,release_date,credits_cast
id,,
421403,1941-09-16,"[{'cast_id': 1, 'character': 'Judge', 'credit_..."
19995,2009-12-10,"[{'cast_id': 242, 'character': 'Jake Sully', '..."
282908,2011-04-30,"[{'cast_id': 0, 'character': 'Michiko Abukumag..."


####VOY A TRAERME OTRO DATASET DE 5000 PELIS DE IMDB PARA MEJORAR LA INFO DE BUDGET Y REVENUE.

In [0]:
#¿Con qué criterio estará la info en este dataset?!!!!

In [0]:
#5000 movies from IMDB.
#Tuve que volver a ejecutar el código de las librerías de Google. Confirmar si he de repetirlo aquí.

file_id = '1ebj6EkGylXTGldfGfJO_AHEAncNf4zKA'

downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("IMDB_5000_movie.csv")


In [43]:
!ls -lh

total 1.5G
-rw-r--r-- 1 root root 2.5K Oct 24 15:15 adc.json
drwx------ 3 root root 4.0K Oct 24 15:19 drive
-rw-r--r-- 1 root root 1.5M Oct 24 17:37 IMDB_5000_movie.csv
-rw-r--r-- 1 root root 1.5G Oct 24 15:15 movies.zip
drwxr-xr-x 2 root root 4.0K Oct 22 18:24 sample_data
drwxr-xr-x 3 root root 4.0K Oct 24 15:16 themoviedb_data


In [44]:
!head IMDB_5000_movie.csv

color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
Color,James Cameron,723,178,0,855,Joel David Moore,1000,760505847,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar ,886204,4834,Wes Studi,0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1,3054,English,USA,PG-13,237000000,2009,936,7.9,1.78,33000
Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,309404152,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End ,471220,48350,Jack Davenport,0,goddess|marriage ceremony|marriage proposal|pirate|singapore,http://www.imdb.com/title/tt0449088/?ref_=fn_tt_tt_1,

In [45]:
my_file_path='./IMDB_5000_movie.csv'

#Reading file and saving in df
dfMovies5000 = pd.read_csv(my_file_path)
dfMovies5000.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [46]:
dfMovies5000.shape

(5043, 28)

In [47]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [48]:
#De aquí tengo que extraer la clave de imdb para relacionarla con dfMovies.
dfMovies5000['movie_imdb_link'].sample(5)

1482    http://www.imdb.com/title/tt0085811/?ref_=fn_t...
4920    http://www.imdb.com/title/tt1702455/?ref_=fn_t...
14      http://www.imdb.com/title/tt1210819/?ref_=fn_t...
4834    http://www.imdb.com/title/tt1314190/?ref_=fn_t...
735     http://www.imdb.com/title/tt0119314/?ref_=fn_t...
Name: movie_imdb_link, dtype: object

In [51]:
#Voy a ver primero de cuántas pelis tengo budget y "gross"
dfMovies5000[['movie_imdb_link', 'gross', 'budget']].sample(5)

,movie_imdb_link,gross,budget
3140,http://www.imdb.com/title/tt1564585/?ref_=fn_t...,21371425.0,10000000.0
1422,http://www.imdb.com/title/tt0120744/?ref_=fn_t...,56876365.0,35000000.0
226,http://www.imdb.com/title/tt0305357/?ref_=fn_t...,100685880.0,120000000.0
1964,http://www.imdb.com/title/tt0117011/?ref_=fn_t...,14095303.0,25000000.0
87,http://www.imdb.com/title/tt0892791/?ref_=fn_t...,238371987.0,165000000.0


In [52]:
dfMovies5000[['movie_imdb_link', 'gross', 'budget']].isnull().any()

movie_imdb_link    False
gross               True
budget              True
dtype: bool

In [53]:
dfMovies5000[['movie_imdb_link', 'gross', 'budget']].count()

movie_imdb_link    5043
gross              4159
budget             4551
dtype: int64

In [0]:
#Cuando vaya a actualizar, estaría bien tener una marca del origen del dato. !!!!!!!!!!!!!!!!!!!!!!!

In [70]:
#Bueno, parece que aunque hay nulos no hay 0's.
dfMovies5000[(dfMovies5000['gross'].notnull()) & (dfMovies5000['gross']>0)].shape

(4159, 28)

In [69]:
#¿Y si exigo que los beneficios sean más de 50$?
dfMovies5000[(dfMovies5000['gross'].notnull()) & (dfMovies5000['gross']>50)].shape

(4159, 28)

In [71]:
#Echo un vistazo de igual modo al budget: estupendo, no hay 0's
dfMovies5000[(dfMovies5000['budget'].notnull()) & (dfMovies5000['budget']>0)].shape

(4551, 28)

In [72]:
#Ni tampoco pelis con presupuesto menor o igual a 50$
dfMovies5000[(dfMovies5000['budget'].notnull()) & (dfMovies5000['budget']>50)].shape

(4551, 28)

In [74]:
#Voy ahora a tratar de extraer el id de IMDB de movie_imdb_link:
dfMovies5000['movie_imdb_link'].sample(5)

4069    http://www.imdb.com/title/tt0053604/?ref_=fn_t...
1061    http://www.imdb.com/title/tt0069293/?ref_=fn_t...
4700    http://www.imdb.com/title/tt2187884/?ref_=fn_t...
3236    http://www.imdb.com/title/tt1247662/?ref_=fn_t...
4099    http://www.imdb.com/title/tt0115632/?ref_=fn_t...
Name: movie_imdb_link, dtype: object

In [86]:
dfMovies5000['movie_imdb_link'].sample(5).str.split("/").map(lambda list: list[4])

633     tt1323594
3309    tt5215952
4579    tt2479800
1541    tt0117333
3252    tt1147687
Name: movie_imdb_link, dtype: object

In [0]:
#Construyo una nueva columna con el identificador de imdb: id_imdb
dfMovies5000['imdb_id'] = dfMovies5000['movie_imdb_link'].str.split("/").map(lambda list: list[4])

In [95]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'id_imdb',
       'imdb_id'],
      dtype='object')

In [96]:
dfMovies5000[['imdb_id', 'movie_imdb_link']].sample(5)

,imdb_id,movie_imdb_link
3115,tt3195644,http://www.imdb.com/title/tt3195644/?ref_=fn_t...
3692,tt0384488,http://www.imdb.com/title/tt0384488/?ref_=fn_t...
2425,tt0081398,http://www.imdb.com/title/tt0081398/?ref_=fn_t...
4503,tt0976025,http://www.imdb.com/title/tt0976025/?ref_=fn_t...
3211,tt1285309,http://www.imdb.com/title/tt1285309/?ref_=fn_t...


In [93]:
#Voy a ver a cuántas pelis en dfMovies podría agregar los revenues:
#CRITERIO 1: tomo el gross de imdb si el revenue de tmdb es 0

#Otra cosa a tener en cuenta: no todas las pelis en dfMovies tenían identificador de imdb:
dfMovies[['imdb_id', 'status']].count()

imdb_id    337380
status     393239
dtype: int64

In [98]:
#Voy a confirmar que no hay duplicados en ninguna de mis tablas:
#(en dfMovies ya lo sé, por cómo descargué la info)
dfMovies5000['imdb_id'].duplicated().sum()
#Hay 124 claves que aparecen más de una vez (sin contar las duplicidades)

124

In [108]:
dfMovies5000[['imdb_id', 'movie_imdb_link']][dfMovies5000['imdb_id'].duplicated()].head()

,imdb_id,movie_imdb_link
137,tt0918940,http://www.imdb.com/title/tt0918940/?ref_=fn_t...
187,tt1673434,http://www.imdb.com/title/tt1673434/?ref_=fn_t...
204,tt4262980,http://www.imdb.com/title/tt4262980/?ref_=fn_t...
303,tt3332064,http://www.imdb.com/title/tt3332064/?ref_=fn_t...
389,tt1502712,http://www.imdb.com/title/tt1502712/?ref_=fn_t...


In [109]:
dfMovies5000[dfMovies5000['imdb_id'] == 'tt0918940']

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,id_imdb,imdb_id
63,Color,David Yates,248.0,110.0,282.0,103.0,Alexander Skarsgård,11000.0,124051759.0,Action|Adventure|Drama|Romance,...,USA,PG-13,180000000.0,2016.0,10000.0,6.6,2.35,29000,tt0918940,tt0918940
137,Color,David Yates,248.0,110.0,282.0,103.0,Alexander Skarsgård,11000.0,124051759.0,Action|Adventure|Drama|Romance,...,USA,PG-13,180000000.0,2016.0,10000.0,6.6,2.35,29000,tt0918940,tt0918940


In [103]:
#Aunque hay 124 códigos imdb_id repetidos, sólo 45 registros registros están repetidos para todos los campos.
dfMovies5000.duplicated().sum()

45

In [104]:
#REalmente, de momento, sólo voy a tomar la info de gross y budget (¿me podría interesar la de los likes???)
dfMovies5000[['imdb_id', 'gross', 'budget']].duplicated().sum()

123

In [0]:
#O sea: los duplicados a nivel de clave lo son también a nivel de gross y budget salvo por un registro. Paso de ese registro y voy a eliminar los
#duplicados quedándome con uno cualquiera de los registros.

In [0]:
dfMovies5000Cleaned = dfMovies5000[['imdb_id', 'gross', 'budget']].drop_duplicates(keep='last')

In [106]:
dfMovies5000.shape

(5043, 30)

In [107]:
dfMovies5000Cleaned.shape

(4920, 3)

In [113]:
#Todavía queda una clave repetida:
dfMovies5000Cleaned['imdb_id'][dfMovies5000Cleaned['imdb_id'].duplicated()]

3967    tt2638144
Name: imdb_id, dtype: object

In [116]:
#Ojo, que lo estoy mirando en la tabla original (para poder coger el título)
dfMovies5000[['imdb_id', 'movie_title', 'gross', 'budget']][dfMovies5000['imdb_id'] == 'tt2638144']
#No me fio para nada del dato del budget. Me cargo el registro.

,imdb_id,movie_title,gross,budget
367,tt2638144,Ben-Hur,NaN,NaN
2613,tt2638144,Ben-Hur,NaN,100000000.0
3967,tt2638144,Ben-Hur,NaN,100000000.0


In [120]:
dfMovies5000Cleaned[['imdb_id', 'gross', 'budget']][dfMovies5000Cleaned['imdb_id'] == 'tt2638144']

,imdb_id,gross,budget
367,tt2638144,NaN,NaN
3967,tt2638144,NaN,100000000.0


In [0]:
 dfMovies5000Cleaned = dfMovies5000Cleaned.drop(labels = [367, 3967], axis=0)

In [124]:
dfMovies5000Cleaned.shape

(4918, 3)

In [0]:
#Y ahora ya puedo cruzar con dfMovies

In [0]:
#dfMovies.merge(dfMovies2, how=’inner’)
#df1.merge(df2, left_on=left_column, right_on=right_column)